#NanoNet
NanoNet is a novel deep learning-based end-to-end modeling tool that given a 
sequence directly produces the 3D coordinates of the Cɑ atoms of the entire VH domain.
It can be used in order to predict structures of nanobodies, VH regions of mAbs and VB regions of TCRs.

The source code and the trained model can be found [here](https://github.com/deepmind/alphafold)

**Note**: If you are predicting many structures in a single run, it is recomanded that you use the option 'write_into_single_pdb_file'. this will write all the predicted structures into a single PDB file and reduce running time and space.

<strong>citecion </strong>

In [ ]:
#@title Input Nb sequence/Nb fasta file (creates a Nb structure for each entry in the fasta file)
from google.colab import files
import re
import os

input_fasta_file = False #@param {type:"boolean"}
input_nb = 'QLVETGGGLVQSGGSLRLSCAASGFTLDNYNIGWFRQAPGKEYGGVSCISSSDGSTYYADSVKGRFTISRDNAKNTVYLQMNNLKPEDTDVYYCAATKYGSSCPIRPYDYWGQGTQVT' #@param {type:"string"}

output_dir = 'NanoNetResutls' #@param {type:"string"}
# remove whitespaces
output_dir = "".join(output_dir.split())
output_dir = re.sub(r'\W+', '', output_dir)


#@markdown ---
#@markdown ### Advanced settings
recunsrtuct_side_chains = False #@param {type:"boolean"}
write_into_single_pdb_file = False #@param {type:"boolean"}
#@markdown ---
#@markdown ### Saving options
save_to_google_drive = False #@param {type:"boolean"}
#@markdown ---
#@markdown Don't forget to hit `Runtime` -> `Run all` after updating form

if not input_fasta_file:  
  # remove whitespaces
  input_nb = "".join(input_nb.split())
  input_nb = re.sub(r'[^a-zA-Z]','', input_nb).upper()
  with open("nb_fasta.fa", "w") as fa_file:
    fa_file.write("> Nb\n")
    fa_file.write("{}\n".format(input_nb))
  input_nb = "nb_fasta.fa"

if not os.path.exists(input_nb):
  print("fasta file {} doesn't exists, aborting.".format(input_nb))

if save_to_google_drive == True:
  from pydrive.drive import GoogleDrive
  from pydrive.auth import GoogleAuth
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  print("Saving results into Drive")

In [ ]:
#@title Clone NanoNet trained model
%%bash

if [ ! -f NanoNetReady ]; then
  # install dependencies
  pip -q install biopython

  # download model
  if [ ! -d "NanoNet/" ]; then
    git clone https://github.com/dina-lab3D/NanoNet --quiet
  fi
  touch NanoNetReady
fi


In [ ]:
#@title Predict Nb structure

#@title Import libraries
if "NanoNet" not in dir():
  from timeit import default_timer as timer
  import sys
  sys.path.insert(0, '/content/NanoNet/')
  from NanoNet import *


start = timer()
run_nanonet(input_nb, "NanoNet/NanoNet", write_into_single_pdb_file, output_dir, False, None)
end = start = timer()

print("NanoNet ended succsufully, total time: {}".format(end-start))

In [ ]:
#@title Download results
#@markdown If you having issues downloading the result zip folder, try disabling your adblocker and run this cell again. If that fails click on the little folder icon to the left, right-click on '\<output_dir>.zip' and select 'Download'.


!zip -FSr $output_dir".zip" $output_dir
files.download(f"{output_dir}.zip")

if save_to_google_drive == True and drive != None:
  uploaded = drive.CreateFile({'title': f"{output_dir}.zip"})
  uploaded.SetContentFile(f"{output_dir}.zip")
  uploaded.Upload()
  print(f"Uploaded {output_dir}.zip to Google Drive with ID {uploaded.get('id')}")